SIDE NOTE: would love to get any critiques, hints, tips, oberservations on any level not just on ML (but obviously ML as the priority).  Beginner Python learner (though I took the Full Stack Course) so any suggestions there are welcome.  Thanks.

Will need to import these libs plus setting some global variables

In [1]:
import sys
import os
import numpy as np
import renders as rs
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from IPython.display import display # Allows the use of display() for DataFrames
from sklearn.externals import joblib
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.cross_validation import StratifiedKFold
from sklearn.svm import SVC
import matplotlib.cm as cm
# Show matplotlib plots inline (nicely formatted in the notebook)
%matplotlib inline

# Might need to change the path of the included libraries.
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/match_stats.py')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/form_model.py')
sys.path.append('/anaconda/envs/stats/lib/python3.5/site-packages')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/model_libs.py')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/form_data.py')
sys.path.append('/Users/senzari/Machine_Learning/stats')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats')
# print(sys.path)
os.chdir('/Users/senzari/Machine_Learning/stats/src')
#print(os.getcwd())

from stats import form_data, match_stats, model_libs, form_model, predict_matches

# Variables
round_number = 27 # for MLS only
target_col = 'points'
ignore_cols = ['match_id', 'team_id', 'team_name', 'opp_id', 'opp_name', 'scheduled', 'games_played', ]
sub_cols = ['current_formation', 'avg_goals_against', 'goal_diff', 'win_percentage', 'sos', 'opp_is_home', 'opp_formation', 'opp_goal_diff',
           'opp_win_percentage', 'opp_sos', 'current_team_yellow_cards', 'current_team_corner_kicks', 'current_team_first_half_goals', 'current_team_sec_half_goals', 
           'opp_team_yellow_cards', 'opp_team_corner_kicks', 'opp_team_first_half_goals', 'opp_team_sec_half_goals']
all_models = ['log', 'svc', 'gmm', 'knn', 'gnb']

""" this variable 'testing' should be false if using CSV's and not pulling from the database. """
testing = False

INITIALIZED...


In [2]:
data_csv = 'round' + str(round_number) + '.csv'

if testing:
    raw_data = form_data.run_data()
    raw_data.to_csv(data_csv)
    print("Raw Data Saved to CSV")
else:
    #Reading in a CSV adds the first index column
    raw_data = pd.read_csv(data_csv)
    raw_data = raw_data.drop(raw_data.columns[[0]], axis=1)

pd.set_option("display.max_columns", 85)

print('Data Loaded...')

Data Loaded...


## Training

Functions that trains basic models (and saves them if need be).  'load_models' will  load exisiting models to save time in testing.

In [3]:
# Removes Columns to Ignore and Splits the Target Column
def split_target(data):
    td = model_libs._clone_and_drop(data, ignore_cols)
    (y, X) = model_libs._extract_target(td, target_col)
    return X, y

"""No Cross-Validation and No Tuning"""
def train_models(round_num, X, y, models):
    
    if os.path.isdir("/models/" + str(round_num)):
        print('Making New Directory for the Round')
        os.chdir('/Users/senzari/Machine_Learning/stats/src/models')
        os.makedirs(str(round_num))
        os.chdir('/Users/senzari/Machine_Learning/stats/src')
        
    finished_models = []
    
    for i in models:
        
        model_round = 'models/' + str(round_num) + '/' + str(i) + '_round_' + str(round_num) + '.pk1'
        
        
        if i == 'log':
            log = form_model.build_model(X, y, i)
            joblib.dump(log, model_round)
            finished_models.append(log)
        elif i == 'svc':
            svc = form_model.build_model(X, y, i)
            joblib.dump(svc, model_round)
            finished_models.append(svc)
        elif i == 'gmm':
            gmm = form_model.build_model(X, y, i)
            joblib.dump(gmm, model_round)
            finished_models.append(gmm)
        elif i == 'knn':
            kmeans = form_model.build_model(X, y, i)
            joblib.dump(kmeans, model_round)
            finished_models.append(kmeans)
        elif i == 'gnb':
            gnb = form_model.build_model(X, y, i)
            joblib.dump(gnb, model_round)
            finished_models.append(gnb)
    
    return finished_models
        
        
def load_models(round_num):
    
    loaded_models = []
    
    for i in all_models:
        model_round = 'models/' + str(round_num) + '/' + str(i) + '_round_' + str(round_num) + '.pk1'
        if i == 'log':
             log = joblib.load(model_round)
             loaded_models.append(log)
        if i == 'svc':
             svc= joblib.load(model_round)
             loaded_models.append(svc)
        elif i == 'gmm':
            gmm = joblib.load(model_round)
            loaded_models.append(gmm)
        elif i == 'knn':
            kmeans = joblib.load(model_round)
            loaded_models.append(kmeans)
        elif i == 'gnb':
            gnb = joblib.load(model_round)
            loaded_models.append(gnb)
        
        print("Success :: Loaded - " + str(i))
        
    return loaded_models

all_data = raw_data

all_data = all_data.drop(sub_cols + ignore_cols + ['goals'], axis=1)

display(all_data.head())

raw_X, raw_y = split_target(all_data)

#log, svc, gmm, knn, gnb = train_models(round_number, raw_X, raw_y)
# svc, gmm, knn, gnb = load_models(round_number)

,is_home,avg_points,avg_goals_for,margin,goal_efficiency,rpi,opp_avg_points,opp_avg_goals,opp_margin,opp_goal_efficiency,opp_rpi,goals_op_ratio,ball_safe_op_ratio,goal_attempts_op_ratio,current_team_possession,current_team_attacks,current_team_dangerous_attacks,current_team_shots_on_target,current_team_shots_total,current_team_ball_safe,current_team_goal_attempts,current_team_saves,current_team_goal_kicks,opp_team_possession,opp_team_attacks,opp_team_dangerous_attacks,opp_team_shots_on_target,opp_team_shots_total,opp_team_ball_safe,opp_team_goal_attempts,opp_team_saves,opp_team_goal_kicks,points
0,0,2.000000,1.333333,-0.333333,0.117647,0.470238,0.666667,0.666667,-1.000000,0.090909,0.584204,2.0,0.904594,1.590909,50.333333,103.333333,60.666667,6.666667,11.666667,85.333333,11.666667,1.666667,5.666667,46.000000,97.666667,50.000000,3.666667,7.333333,94.333333,7.333333,1.333333,7.666667,3
1,1,2.000000,1.666667,0.000000,0.333333,0.386905,0.333333,0.666667,-0.666667,0.000000,0.504202,2.5,1.145161,0.820513,50.666667,112.000000,56.333333,5.666667,10.666667,94.666667,10.666667,1.666667,5.333333,57.000000,96.333333,64.666667,6.000000,12.333333,82.666667,13.000000,4.000000,5.000000,1
2,1,2.333333,2.000000,1.666667,0.052632,0.562147,1.333333,1.333333,-0.333333,0.125000,0.630952,1.5,0.928571,2.368421,49.333333,113.000000,56.000000,6.333333,15.000000,95.333333,15.000000,3.000000,5.666667,48.333333,103.000000,50.333333,3.666667,6.333333,102.666667,6.333333,2.333333,10.333333,1
3,0,1.666667,2.000000,1.000000,0.153846,0.518304,0.666667,1.333333,-1.000000,0.333333,0.531369,1.5,0.989011,1.708333,51.333333,103.333333,48.333333,5.666667,13.666667,90.000000,13.666667,2.666667,5.333333,47.666667,100.000000,54.666667,3.666667,8.000000,91.000000,8.000000,3.000000,6.333333,3
4,1,1.666667,2.000000,1.000000,0.153846,0.602431,2.000000,1.333333,0.666667,0.285714,0.598333,1.5,0.912162,1.863636,51.333333,103.333333,48.333333,5.666667,13.666667,90.000000,13.666667,2.666667,5.333333,47.666667,105.666667,50.666667,3.000000,7.333333,98.666667,7.333333,5.333333,10.666667,3


In [4]:
def run_features(data, drop_data, target, models):
    
    new_data = data.drop(drop_data, axis=1)
    
    display(new_data.head())
    
    (y, X) = model_libs._extract_target(new_data, target)
    
    models = train_models(round_number, X, y, models)
    
    return models


#### Running ALL Features 
models_test_1 = run_features(all_data, [], 'points', all_models)

,is_home,avg_points,avg_goals_for,margin,goal_efficiency,rpi,opp_avg_points,opp_avg_goals,opp_margin,opp_goal_efficiency,opp_rpi,goals_op_ratio,ball_safe_op_ratio,goal_attempts_op_ratio,current_team_possession,current_team_attacks,current_team_dangerous_attacks,current_team_shots_on_target,current_team_shots_total,current_team_ball_safe,current_team_goal_attempts,current_team_saves,current_team_goal_kicks,opp_team_possession,opp_team_attacks,opp_team_dangerous_attacks,opp_team_shots_on_target,opp_team_shots_total,opp_team_ball_safe,opp_team_goal_attempts,opp_team_saves,opp_team_goal_kicks,points
0,0,2.000000,1.333333,-0.333333,0.117647,0.470238,0.666667,0.666667,-1.000000,0.090909,0.584204,2.0,0.904594,1.590909,50.333333,103.333333,60.666667,6.666667,11.666667,85.333333,11.666667,1.666667,5.666667,46.000000,97.666667,50.000000,3.666667,7.333333,94.333333,7.333333,1.333333,7.666667,3
1,1,2.000000,1.666667,0.000000,0.333333,0.386905,0.333333,0.666667,-0.666667,0.000000,0.504202,2.5,1.145161,0.820513,50.666667,112.000000,56.333333,5.666667,10.666667,94.666667,10.666667,1.666667,5.333333,57.000000,96.333333,64.666667,6.000000,12.333333,82.666667,13.000000,4.000000,5.000000,1
2,1,2.333333,2.000000,1.666667,0.052632,0.562147,1.333333,1.333333,-0.333333,0.125000,0.630952,1.5,0.928571,2.368421,49.333333,113.000000,56.000000,6.333333,15.000000,95.333333,15.000000,3.000000,5.666667,48.333333,103.000000,50.333333,3.666667,6.333333,102.666667,6.333333,2.333333,10.333333,1
3,0,1.666667,2.000000,1.000000,0.153846,0.518304,0.666667,1.333333,-1.000000,0.333333,0.531369,1.5,0.989011,1.708333,51.333333,103.333333,48.333333,5.666667,13.666667,90.000000,13.666667,2.666667,5.333333,47.666667,100.000000,54.666667,3.666667,8.000000,91.000000,8.000000,3.000000,6.333333,3
4,1,1.666667,2.000000,1.000000,0.153846,0.602431,2.000000,1.333333,0.666667,0.285714,0.598333,1.5,0.912162,1.863636,51.333333,103.333333,48.333333,5.666667,13.666667,90.000000,13.666667,2.666667,5.333333,47.666667,105.666667,50.666667,3.000000,7.333333,98.666667,7.333333,5.333333,10.666667,3


Training LOG REG Model
Score on Training Set :: 0.560606060606
Score on Test Set :: 0.36
Finished LOG REG Modeling
-----------------------------------
Training SVC Model
F1 score for training set: 0.586846599417
F1 score for test set: 0.413516104429
Finished SVC Modeling
-----------------------------------
Training GMM Modeling
Silhouette Score :: 0.165289928353 for 2 Clusters
Silhouette Score :: 0.0406623366899 for 3 Clusters
Finished GMM Modeling
-----------------------------------
Training K Neighbors Classifier Model
KNN Score :: 0.651209677419
Finished K-Means Modeling
-----------------------------------
Training Gaussian NB Model
F1 score for training set: 0.474656338511
F1 score for test set: 0.336475869809
Finished Gaussian NB Modeling


### STEP 1 - Tune Models

Tune models using their respective parameters if they have some.

In [ ]:
def train_tuned_models(round_num, X, y):
    for i in models:
        form_model.build_tuned_model(X, y, i)
        
train_tuned_models(round_number, raw_X, raw_y)

### STEP 2 - Optimizing Features
Using RFECV to see which features are the most pertinent.  Can only use SVC with RFECV.

In [ ]:
def plot_RFECV(X, y):
    svc = SVC(kernel="linear")
    rfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(y, 2),
                  scoring='accuracy')
    rfecv.fit(X, y)

    # Plot number of features VS. cross-validation scores
    plt.figure()
    plt.xlabel("Number of features selected")
    plt.ylabel("Cross validation score (nb of correct classifications)")
    plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
    plt.show()
    
plot_RFECV(raw_X, raw_y)

Using the RFE's ranking system we will use only the top 'x' number of features to try to find the optimial number of features (for SVC).  'X' is roughly determined by the number of features outputted in the RFECV plot.

In [ ]:
def create_RFE(X, y):
    # Create the RFE object and rank each pixel
    svc = SVC(kernel="linear", C=1)
    rfe = RFE(estimator=svc, n_features_to_select=1, step=1)
    rfe.fit(X, y)
    print(rfe.ranking_)
    return rfe.ranking_
   
rankings = create_RFE(raw_X, raw_y)

### STEP 3
Take ranked features and reordering the features so that we can use that DF in the new models

In [ ]:
def reorder_features(ranking, data):
    ranked_X = pd.DataFrame(index=range(480))
    for a in range(len(ranking)):
        for i in range(len(ranking)):
            if ranking[i] == a+1:
                ranked_X = ranked_X.join(data.ix[:,i])
    
    display(ranked_X.describe())
    return ranked_X


ranked_X = reorder_features(rankings, raw_X)

### STEP 4
Let's try to reduce the number of features to at least 25 although optimal seems likely around ~8 features.  Note: Even though RFECV only applies to SVC still running it on the other models just to see the impact.

In [ ]:
def train_ranked_models(X, y):
    
    for x in xrange(5, 25):
        print('X :: {}'.format(x))
        svc, gmm, knn, gnb = train_models(round_number, X.iloc[:, 1:x], y)
        
train_ranked_models(ranked_X, raw_y)

Plot PCA based off of optimized number in previous step

### STEP 5 Try PCA on ranked features 

In [ ]:
def plot_PCA(data):
    pca = PCA(n_components=7)
    pca.fit(data)
    # Generate PCA results plot
    pca_results = rs.pca_results(data, pca)
    return pca

pca = plot_PCA(ranked_X)

In [ ]:
def apply_PCA(pca, data):
    # TODO: Apply a PCA transformation the good data
    pca_data = pca.transform(data)
    return pca_data
    
apply_PCA(pca, ranked_X.iloc[:, 1:23])

## Part 2
Run all the above steps but using standard features and the Home/Away Features - Create Home/Away Data.  Home/Away Data is my attempt to try to split out the difference in how a current team's play varies from away and home.  For instance, instead of just averaging a teams goal scored from all games, average a teams goal scored on home games and away games.  If in that match they are the home team, only use the home average.  If they are away, use the away average.

Using desired model to predict upcoming matches :: Right now these are the best performing models with the best performing feature.  Seems just removing ties on the 'raw data' has the best impact 

In [ ]:
print('\n :: Removing Ties on Raw Data :: ')
X, y = remove_ties(raw_data)
print('\n :: Run Training on Removing Ties from the Raw Data :: ')
svc, gmm, knn, gnb = train_models(round_number, X, y)

### Part 4

Use these models trained on the optimal data to predict upcoming matches

Find upcoming matches...

In [ ]:
#upcoming_matches, match_details = predict_matches.get_upcoming_matches()
#upcoming_matches.to_csv('upcoming_matches.csv')
#match_details.to_csv('match_details.csv')
#print("Upcoming Data Saved to CSV")

upcoming_matches = pd.read_csv('upcoming_matches.csv')
upcoming_matches = upcoming_matches.drop(upcoming_matches.columns[[0]], axis=1)

match_details = pd.read_csv('match_details.csv')
match_details = match_details.drop(match_details.columns[[0]], axis=1)

display(upcoming_matches)

Make predictions on upcoming data

In [ ]:
from stats import predict_matches

for i in models:
    if i == 'svc':
        svc_preds, upcoming_data = predict_matches.predictions(upcoming_matches, match_details, svc)
        print(svc_preds)
    elif i == 'gmm':
        gmm_preds, upcoming_data = predict_matches.predictions(upcoming_matches, match_details, gmm)
        print(gmm_preds)
    elif i == 'knn':
        knn_preds, upcoming_data = predict_matches.predictions(upcoming_matches, match_details, knn)
        print(knn_preds)
    elif i == 'gnb':
        gnb_preds, upcoming_data = predict_matches.predictions(upcoming_matches, match_details, gnb)
        print(gnb_preds)
        
display(upcoming_data.head())

Output those predictions to a CSV so easier to visualize

In [ ]:
columns = ['team_name', 'opp_name', 'scheduled']
# Remove all columns except the ones above
upcoming_matches = upcoming_data[columns]

# Add predictions to the end of that DF
results = pd.DataFrame({'SVC': svc_preds, 'GMM': gmm_preds, 'KNN': knn_preds, 'GNB': gnb_preds})
upcoming_matches = upcoming_matches.join(results)
reordered_matches = pd.DataFrame([])

for rows in upcoming_matches.iterrows():
    for i in upcoming_matches['team_name']:
        if rows[1]['opp_name'] == i:
            reordered_matches = reordered_matches.append(rows[1])
            reordered_matches = reordered_matches.append(upcoming_matches[upcoming_matches['team_name'].isin([i])])

reordered_matches = reordered_matches.drop_duplicates() 
columns = ['scheduled', 'team_name', 'opp_name', 'SVC', 'KNN', 'GMM', 'GNB']
reordered_matches = reordered_matches[columns]
# upcoming_matches = upcoming_matches[(upcoming_matches['scheduled'] < '2016-08-26')]
reordered_matches.to_csv('predictions_' + str(round_number) + '.csv')
print('Prediction CSV saved')

### RESULTS

Excluding GMM...

FC Dallas vs Portland Timbers :: 3-1 (Unanimously Dallas)
New England Revolution vs Colorado Rapids :: 2-0 (Mixed, 1 model picked NER to win)
New York City FC vs DC United :: 3-2 (Unanimously NYCFC)
Chicago Fire vs Philadelphia Union :: 3-0 (2 models picked Chi and 1 picked Phi)
Vancouver Whitecaps vs New York Red Bulls :: 0-1 (WRONG, 2 models picked Vancouver to win)
LA Galaxy vs Columbus Crew :: 2-1 (2 models picked LA to win)

My Questions:
- For GMM, how can we tell what classification it's predicting.  If you notice in the predictions CSV it says 0, 1, 2.  How do I know which one of those are win, draw, or a loss (3, 1, 0)?